# Mount Google Drive

In [1]:
# from google.colab import drive
# drive.mount("/content/drive/")

In [2]:
# !pip3 install fse
# !pip3 install pytorch-nlp

# conda install -c anaconda nltk
#  conda install -c anaconda pandas
# conda create -n sklearn-env -c conda-forge scikit-learn
# conda activate sklearn-env
# conda install pytorch torchvision torchaudio -c pytorch

# Import Libraries

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import re
import string
import collections
import fse
import random

import random
import math
import time

import itertools
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchnlp.metrics import get_moses_multi_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)

import nltk
nltk.download('punkt')

import nltk
from nltk.util import ngrams
nltk.download('punkt')
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

# Any results you write to the current directory are saved as output.

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nurrizkyimani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nurrizkyimani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# !wget "https://storage.googleapis.com/babert-pretraining/IndoNLU_finals/models/fasttext/fasttext.4B.id.300.epoch5.uncased.bin" -P "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer"

# Importing Dataset GDrive

In [5]:
# # import dataset from google drive
# path = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/annotated/combined/csv/"
# all_agree_path = path + "main.csv"
# all_agree_pd = pd.read_csv(all_agree_path)
# df_clickbait = all_agree_pd[all_agree_pd['label'] == "clickbait"]
# df_nonclickbait = all_agree_pd[all_agree_pd['label'] == "non-clickbait"]
# df_both = pd.concat((df_clickbait, df_nonclickbait), ignore_index=True)
#
# df_both_copy = df_both.copy()
# df_both_copy['label'] = df_both_copy['label'].astype('category')
# df_both_copy.head(-1)

# Importing dataset

In [81]:
# Import the csv
all_agree_path = "2022_thesis_styletransfer/annotated/combined/csv/main.csv"
all_agree_pd = pd.read_csv(all_agree_path)
df_clickbait = all_agree_pd[all_agree_pd['label'] == "clizckbait"]
df_nonclickbait = all_agree_pd[all_agree_pd['label'] == "non-clickbait"]
df_both = pd.concat((df_clickbait, df_nonclickbait), ignore_index=True)

df_both_copy = df_both.copy()
df_both_copy['label'] = df_both_copy['label'].astype('category')
df_both_copy.head(-1)

,title,label,label_score
0,"Masuk Radar Pilwalkot Medan, Menantu Jokowi Be...",non-clickbait,0
1,Malaysia Sudutkan RI: Isu Kabut Asap hingga In...,non-clickbait,0
2,"Kemensos Salurkan Rp 7,3 M bagi Korban Kerusuh...",non-clickbait,0
3,"Terkait Mayat Bayi Mengenaskan di Tangerang, S...",non-clickbait,0
4,"Uang Rp 1,6 M yang Raib di Parkiran Kantor Gub...",non-clickbait,0
...,...,...,...
8704,"Diperankan Bastian Bintang, ANTV Bakal Hadirka...",non-clickbait,0
8705,Gubernur Tetapkan Riau Darurat Pencemaran Udar...,non-clickbait,0
8706,"Asap Karhutla Riau Mulai Merambah Ke Nias, BM...",non-clickbait,0
8707,"Tolak RUU Pertanahan, Ribuan Petani Siap Gelar...",non-clickbait,0


In [7]:
df_both_label = df_both_copy.drop('label_score', axis=1)
df_both = df_both_copy.drop('label', axis=1)
df_both.head(-5)

,title,label_score
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,1
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",1
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,1
3,Detik-detik Lima Kendaraan Alami Kecelakaan Be...,1
4,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,1
...,...,...
14990,"Wamena Papua Kembali Membara, Kantor Bupati Di...",0
14991,BMKG Angkat Bicara Soal Langit Merah Jambi,0
14992,Densus 88 Tangkap Teroris Jaringan ISIS di Jak...,0
14993,Operasional Bandara Wamena Dihentikan Akibat R...,0


In [8]:
len(df_both)

15000

In [9]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
df_both_copy['label_encoding'] = labelencoder.fit_transform(df_both_copy['label'])
df_encode_clean = df_both_copy.copy()
df_encode_clean = df_both = df_both_copy.drop(['label_score'], axis=1)
df_encode_clean.head()

,title,label,label_encoding
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,0
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",clickbait,0
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,clickbait,0
3,Detik-detik Lima Kendaraan Alami Kecelakaan Be...,clickbait,0
4,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,clickbait,0


# Preprocessing : Lower Casing

In [10]:
# preprocessing : lower casing
df_encode_clean['title'] = df_encode_clean['title'].str.lower()
df_encode_clean.title

0        viral! driver ojol di bekasi antar pesanan mak...
1        ada motor nyangkut di atas bambu di sleman, ko...
2             pesan gamblang poyuono menolak revisi uu kpk
3        detik-detik lima kendaraan alami kecelakaan be...
4        kocak! maling di rumah mewah jakut terekam cct...
                               ...                        
14995    gubernur tetapkan riau darurat pencemaran udar...
14996     asap karhutla riau mulai merambah ke nias, bm...
14997    tolak ruu pertanahan, ribuan petani siap gelar...
14998    beredar isu internet papua diblokir lagi, telk...
14999    emmy awards 2019: peter dinklage cetak rekor b...
Name: title, Length: 15000, dtype: object

Convert the titl into lower case for all label

In [11]:
# Add space between punctuation
df_encode_clean_t =  df_encode_clean.copy()
df_encode_clean_t['title'] = df_encode_clean_t['title'].apply(lambda y: " ".join((re.sub(r'([!/?/./,/%/\'/\"/-/:])', lambda x: ' ' + x.group()+' ' , y)).split()) , 1)
df_encode_clean_t

,title,label,label_encoding
0,viral ! driver ojol di bekasi antar pesanan ma...,clickbait,0
1,"ada motor nyangkut di atas bambu di sleman , k...",clickbait,0
2,pesan gamblang poyuono menolak revisi uu kpk,clickbait,0
3,detik-detik lima kendaraan alami kecelakaan be...,clickbait,0
4,kocak ! maling di rumah mewah jakut terekam cc...,clickbait,0
...,...,...,...
14995,gubernur tetapkan riau darurat pencemaran udar...,non-clickbait,1
14996,"asap karhutla riau mulai merambah ke nias , bm...",non-clickbait,1
14997,"tolak ruu pertanahan , ribuan petani siap gela...",non-clickbait,1
14998,"beredar isu internet papua diblokir lagi , tel...",non-clickbait,1


In [12]:
# separate between clickbait clean and nonclickbait clean
df_clickbait_clean = df_encode_clean_t[df_encode_clean_t['label'] == "clickbait"]
df_nonclickbait_clean = df_encode_clean_t[df_encode_clean_t['label'] == "non-clickbait"]

In [13]:
# create a list form the combination of df_clickbait and nonclickbait

d_both = pd.concat((df_clickbait_clean, df_nonclickbait_clean ), ignore_index=True)
d_both = d_both.drop(['label', 'label_encoding'], axis=1)
d_both = d_both.values.tolist()
d_both = list(itertools.chain.from_iterable(d_both))

In [14]:
d_both_clean = pd.concat((df_clickbait_clean, df_nonclickbait_clean ), ignore_index=True)
d_both_clean = d_both_clean.drop(['label'], axis=1)
d_both_clean

,title,label_encoding
0,viral ! driver ojol di bekasi antar pesanan ma...,0
1,"ada motor nyangkut di atas bambu di sleman , k...",0
2,pesan gamblang poyuono menolak revisi uu kpk,0
3,detik-detik lima kendaraan alami kecelakaan be...,0
4,kocak ! maling di rumah mewah jakut terekam cc...,0
...,...,...
14995,gubernur tetapkan riau darurat pencemaran udar...,1
14996,"asap karhutla riau mulai merambah ke nias , bm...",1
14997,"tolak ruu pertanahan , ribuan petani siap gela...",1
14998,"beredar isu internet papua diblokir lagi , tel...",1


In [15]:
d_both

['viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda',
 'ada motor nyangkut di atas bambu di sleman , kok bisa ?',
 'pesan gamblang poyuono menolak revisi uu kpk',
 'detik-detik lima kendaraan alami kecelakaan beruntun di cipularang',
 'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur',
 'hampir seluas jabar , ini tantangan sintang siapkan generasi emas',
 'viral video diduga baku tembak di sleman , ini kata polisi',
 'waspada ! ada penipuan catut pertamina , korbannya rugi puluhan juta',
 'jaksa beberkan senpi-peluru tajam pembelian kivlan zen cs , ini rinciannya',
 'sering quality time bersama keluarga ? ternyata ini 3 manfaatnya !',
 'menteri jokowi 55 % profesional , ini kandidatnya ?',
 'sebut pemilihan capim kpk cacat yuridis , ini saran samad ke dpr',
 "pria misterius berjubah putih viral di sumut , ingatkan ' tuhan murka '",
 'video porno di sumedang disebar pemeran pria , apa motifnya ?',
 'terungkap ! video porno di sumedang diperankan p

In [16]:
# create list of clickbait and nonclickbait headline

df_clickbait_t = df_clickbait_clean.drop(['label', 'label_encoding'], axis=1).values.tolist()
clickbait_l_c = list(itertools.chain.from_iterable(df_clickbait_t))

df_nonclickbait_t = df_nonclickbait_clean.drop(['label', 'label_encoding'], axis=1).values.tolist()
nonclickbait_l_c = list(itertools.chain.from_iterable(df_nonclickbait_t))

print(nonclickbait_l_c[0:5])
print(clickbait_l_c[0:5])

['masuk radar pilwalkot medan , menantu jokowi bertemu dpw nasdem sumut', 'malaysia sudutkan ri : isu kabut asap hingga invasi babi', 'kemensos salurkan rp 7 , 3 m bagi korban kerusuhan sosial di papua', 'terkait mayat bayi mengenaskan di tangerang , seorang pria ditangkap polisi', 'uang rp 1 , 6 m yang raib di parkiran kantor gubernur sumut untuk honor tapd']
['viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda', 'ada motor nyangkut di atas bambu di sleman , kok bisa ?', 'pesan gamblang poyuono menolak revisi uu kpk', 'detik-detik lima kendaraan alami kecelakaan beruntun di cipularang', 'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur']


# Feature Extraction: Ngram [100% DONE]

In [17]:
#these are methods that will become useful when extracting attribute markers
#why do we need all this? well... that's like 5 hours of debugging...
def flatten(foo):
    return list(_flatten(foo))

def _flatten(foo):
    for x in foo:
        if isinstance(x, collections.Iterable) and not isinstance(x, str):
            for y in _flatten(x):
                yield y
        else:
            yield x

def array_to_string(a):
    return ' '.join(flatten(a))

def is_in_string_array(elements, original): #deprecated, does not take into account sequence order
    return np.isin(array_to_string(elements).split(), array_to_string(original).split()).any()
# no usage in the func
def insert_string(string, inserted_string, index):
    return string[:index] + inserted_string + string[index:]

# modified from https://stackoverflow.com/questions/41752946/replacing-a-character-from-a-certain-index
def replace_string(s, newstring, index, nofail=False):
    # raise an error if index is outside of the string
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string. index:" + index)

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + len(newstring):]

In [18]:
# creating function for making ngram from list of headline
def ngram_maker_v2(list_sentences, min_length, max_length):
    lengths = range(min_length, max_length + 1)
    res_ngram = {length : [] for length in lengths}

    for leng in lengths:
        for sentence in list_sentences:
            temp_set = set()
            n_grams = ngrams(nltk.word_tokenize(sentence), leng)

            for grams in n_grams:
                res = ' '.join(grams)
                temp_set.add(res)

            for key in temp_set:
                split_t = tuple(key.split(" "))
                res_ngram[leng].append(split_t)

    return res_ngram

In [19]:
# create a function that count the ngram occurance
def counter_for_ngram(ngram_dict, min_length, max_length):
    lengths = range(min_length, max_length + 1)
    ngram_count_res = {length : [] for length in lengths}

    for key in ngram_dict:
        ng_count = Counter(ngram_dict[key])
        ngram_count_res[key] = ng_count

    return ngram_count_res

In [20]:
param_span_t = 3
clickbait_ngrams_v2= ngram_maker_v2(list_sentences=clickbait_l_c, min_length=1, max_length=param_span_t)
clickbait_ngram_count_v2 = counter_for_ngram(ngram_dict=clickbait_ngrams_v2,min_length=1, max_length=param_span_t)

non_clickbait_ngrams_v2= ngram_maker_v2(list_sentences=nonclickbait_l_c, min_length=1, max_length=param_span_t)
non_clickbait_ngram_count_v2 = counter_for_ngram(ngram_dict=non_clickbait_ngrams_v2,min_length=1, max_length=param_span_t)

In [21]:
# creat function for get ngram from the word in the one sentence reference only
def ngram_from_sentence_v3(ori_sentence, min_length, max_length):
    sent_in_arr = [ori_sentence]
    res_ngram_sentence = []
    stc_ngram_only = ngram_maker_v2(list_sentences=sent_in_arr, min_length=min_length, max_length=max_length)
    return stc_ngram_only

def count_ngram_sentence_onlyV3(ngram_dictionary, ngram_from_sentence, min_length, max_length):
    lengths = range(min_length, max_length + 1)
    ngram_compare_arr = []

    for length in lengths:
        for split in ngram_from_sentence[length]:
            ngram_compare_arr.append([split, ngram_dictionary[length][split]])

    return np.array(ngram_compare_arr)

In [22]:
# create function for get the attribute of the sentence

smoothing_parameter = 1

def array_to_string(a):
    return ' '.join(flatten(a))

# the word is in the final array;
def is_in_string_array_v2(elements, original):
    return np.isin(array_to_string(elements).split(), array_to_string(original).split()).any()

# the the attribute based on the sentence, with min_length, max_length and the reference clickbait/nonclickbait dictinoary
def get_attribute_sentence(sentence, min_length, max_length, ref_bait, param_threshold):
    # ngram sentence as reference
    sentence_ngram_ref = ngram_from_sentence_v3(ori_sentence=sentence, min_length=min_length, max_length=max_length)

    # count the clickbait sentence, clickbait ngram count as based reference
    counts_sentence_clickbait = count_ngram_sentence_onlyV3(clickbait_ngram_count_v2,sentence_ngram_ref,min_length,max_length)

    # print(counts_sentence_clickbait)

    # count the non-clickbait sentence, non-clickbait ngram count as based reference
    counts_sentence_nonclickbait = count_ngram_sentence_onlyV3(non_clickbait_ngram_count_v2,sentence_ngram_ref,min_length,max_length)

    importances = 0

    csn_int = counts_sentence_nonclickbait[:,1]
    csn_ngram = counts_sentence_nonclickbait[:,0]

    csc_int = counts_sentence_clickbait[:,1]
    csc_ngram = counts_sentence_clickbait[:,0]

    if ref_bait == "to-clickbait":
        top = (csc_int + smoothing_parameter)
        bottom = (csn_int + smoothing_parameter)
        importances = (top/bottom)
    elif ref_bait == "to-non-clickbait":
        top = (csn_int+ smoothing_parameter)
        bottom = (csc_int + smoothing_parameter)
        importances = (top/bottom)

    # print(counts_sentence_clickbait)

    de = counts_sentence_clickbait[:, 0]
    # print("impot: ", importances)

    # print(" de: ", de)
    importances_t = np.vstack((importances, de)).T

    # print("importances_t", importances_t)

    attribute_words = []
    for importance in importances_t:
        if importance[0] > param_threshold and not is_in_string_array_v2(importance[1], attribute_words):
            # print(importance[1])
            attribute_words.append(' '.join(importance[1]))

    return attribute_words

In [23]:
# run the get attribute sentence
test11 = get_attribute_sentence("viral ini ! bj habibie meninggal", 1, 2, "to-clickbait", 4 )

/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_2115/4054946038.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)
/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_2115/1295762271.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(x, collections.Iterable) and not isinstance(x, str):


In [24]:
def separate(sentence, style_src, param_threshold):
    attributes = get_attribute_sentence(sentence, 1, 2, "to-clickbait", param_threshold )
    c = sentence

    replace_indexes = []

    for a in attributes:
        replace_index = -1
        replace_index = c.find(a)
        replace_indexes.append(replace_index)
        c = c.replace(a, " "*len(a))

    if len(attributes) == 0:
        return {'c': c, 'a': [], 'i': [], 's': sentence}

    replace_indexes, attributes = zip(*sorted(zip(replace_indexes, attributes)))
    return {'c': c, 'a': attributes, 'i': replace_indexes, 's': sentence}

def get_c(sentence, style, param_threshold):
    return re.sub(' +', ' ', separate(sentence, style, param_threshold)['c'])

def get_a(sentence, style, param_threshold):
    a = separate(sentence, style, param_threshold)['a']
    if len(a) > 0:
        return ' '.join(a)
    else:
        return ""

# Preprocessed : TFIDF, Retrieve Function [100% DONE]

to get similar sentece in the clickbait headline list

In [25]:
# 3 of all headline list
# d_both
# nonclickbait_l_c
# clickbait_l_c

In [24]:
# import all needed librarty
tfidf_vectorizer=TfidfVectorizer(use_idf=True, stop_words=None)
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(d_both)
dict_idf = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer.idf_))

/Users/nurrizkyimani/.conda/envs/workspace/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [25]:
param_backoff_limit = 3
def get_overlap(a, b):
    #     print(a, b)
    a_counter = collections.Counter(a.split())
    b_counter = collections.Counter(b.split())
    overlap = a_counter & b_counter
    return overlap

def get_weighted_overlap(a, b):
    overlap = get_overlap(a, b)
    a_counter = collections.Counter(a.split())
    #calculate
    weighted_overlap = 0
    for word in overlap:

        word_tf = a_counter[word]#/len(a.split()) -> commented out cause division by constant value doesn't matter

        get_idf = dict_idf.get(word)
        word_idf = 1 if get_idf == None else get_idf #get rid of error when idf not in dict

        word_tfidf = word_tf*word_idf
        weighted_overlap+=overlap[word]*word_tfidf

    return weighted_overlap

# 0 = nonclickbait_l_c
# 1 = clickbait_l_c

def get_similar_sentence_v2(sentence, style_src):
    opposite_dataset = clickbait_l_c if style_src else nonclickbait_l_c

    closest_sentence = ""
    highest_overlap = 0
    previous_sentences = []

    for sentence_b in opposite_dataset:
        weighted_overlap = get_weighted_overlap(sentence, sentence_b)
        if weighted_overlap > highest_overlap and sentence_b not in previous_sentences:
            highest_overlap = weighted_overlap
            closest_sentence = sentence_b

    return closest_sentence

def get_similar_sentence(sentence, style_src):

    opposite_dataset = clickbait_l_c if style_src else nonclickbait_l_c
    highest_overlap = 0
    closest_sentence = ""
    num_markers = 0
    previous_sentences = []
    backoff_count = 0

    min_attribute_markers=len(get_attribute_sentence(sentence, 1, 2, "to-non-clickbait", 5 ))

    while(num_markers < min_attribute_markers and backoff_count < param_backoff_limit):
        for sentence_b in opposite_dataset:
            weighted_overlap = get_weighted_overlap(sentence, sentence_b)
            if weighted_overlap > highest_overlap and sentence_b not in previous_sentences:
                highest_overlap = weighted_overlap
                closest_sentence = sentence_b
        highest_overlap = 0
        backoff_count += 1
        previous_sentences.append(closest_sentence)
        num_markers = len((get_attribute_sentence(closest_sentence, 1, 2, "to-clickbait", 5 )))

    return closest_sentence

In [26]:
# Retrieve using tfidf
def retrieve_v2(sentence, style_ret_src):
    style_src = 0
    if(style_ret_src == "to-clickbait"):
        style_src = 1

    similar_stc =  get_similar_sentence_v2(sentence=sentence, style_src=style_src)
    print("v2 stc: ", similar_stc)
    return separate(sentence=similar_stc, style_src= "to-clickbait", param_threshold=5)

# Retrieve using tfidf
def retrieve(sentence, style_ret_src):
    style_src = 0
    if(style_ret_src == "to-clickbait"):
        style_src = 1

    similar_stc =  get_similar_sentence(sentence=sentence, style_src=style_src)
    print("v1 stc: ", similar_stc)
    return separate(sentence=similar_stc, style_src= "to-clickbait", param_threshold=5)

In [27]:
stc_test_retrieve = "tim jibom disposal bahan peledak di rumah terduga teroris di jakut"
retrieve(stc_test_retrieve, "to-clickbait")
# retrieve_v2(stc_test_retrieve,  "to-clickbait")

v1 stc:  duarr ! bahan peledak terduga teroris jakut didisposal , warga tepuk tangan


/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_46291/4054946038.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)


{'c': 'duarr   bahan peledak terduga teroris jakut didisposal , warga tepuk tangan',
 'a': ('!',),
 'i': (6,),
 's': 'duarr ! bahan peledak terduga teroris jakut didisposal , warga tepuk tangan'}

# Feature Extraction: Generate, Dictionary Preprocess [100%]

In [30]:
# Resources: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# Had to modify & adapt most of the code in the tutorial since this isn't translation & data preprocessing is different

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [31]:
# count the overlap; we need function for this;
def get_overlap(a, b):
    a_counter = collections.Counter(a.split())
    b_counter = collections.Counter(b.split())
    overlap = a_counter & b_counter
    return overlap

def get_total_overlap(a, b):
    return len(list(get_overlap(a, b).elements()))

In [32]:

noise_chance = 0.4
param_t = 4

def prepareData():
    input_lang = Lang("input")
    output_lang = Lang("output")

    d_pos_a = []
    pairs_pos = []

    for sentence in nonclickbait_l_c:
        c = get_c(sentence, 1, param_t)
        a = get_a(sentence, 1, param_t)
        d_pos_a.append(a)
        pairs_pos.append([c, a, sentence])

    d_neg_a = []
    pairs_neg = []

    for sentence in clickbait_l_c:
        c = get_c(sentence, 0, param_t)
        a = get_a(sentence, 0, param_t)
        d_neg_a.append(a)
        pairs_neg.append([c, a, sentence])

    #adding noise for pos
    for pair in pairs_pos:
        if random.random() < noise_chance:
            real_a = pair[1].split()

            if(len(real_a) == 0):
                continue

            for a in d_pos_a:
                if(len(a) == 0):
                    continue
                a = a.split()
                overlap = get_total_overlap(' '.join(real_a), ' '.join(a))
                if overlap > 0 and ((overlap == len(real_a) - 1 and len(real_a) - len(a) == 1) or (overlap == len(real_a) and len(real_a) - len(a) == -1)):
                    real_a = a
                    break;
            pair[1] = ' '.join(real_a)

    #adding noise for neg
    for pair in pairs_neg:
        if random.random() < noise_chance:
            real_a = pair[1].split()
            if(len(real_a) == 0):
                continue

            for a in d_neg_a:
                if(len(a) == 0):
                    continue
                a = a.split()

                overlap = get_total_overlap(' '.join(real_a), ' '.join(a))
                if overlap > 0 and ((overlap == len(real_a) - 1 and len(real_a) - len(a) == 1) or (overlap == len(real_a) and len(real_a) - len(a) == -1)):
                    real_a = a
                    break;
            pair[1] = ' '.join(real_a)

    pairs = np.concatenate((pairs_pos, pairs_neg), 0)
    for pair in pairs:
        input_lang.addSentence(pair[0])
        input_lang.addSentence(pair[1])
        output_lang.addSentence(pair[2])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs, pairs_neg, pairs_pos

input_lang, output_lang, pairs, pairs_click, pairs_nonclick = prepareData()

pairs_click

/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_45599/4054946038.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)


Counted words:
input 18304
output 18268


[[' driver di bekasi antar pesanan makanan pakai sepeda',
  'viral ! ojol',
  'viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda'],
 ['ada motor nyangkut bambu di sleman , ',
  'di atas kok bisa ?',
  'ada motor nyangkut di atas bambu di sleman , kok bisa ?'],
 ['pesan gamblang poyuono menolak revisi uu kpk',
  '',
  'pesan gamblang poyuono menolak revisi uu kpk'],
 [' lima kendaraan alami kecelakaan beruntun di cipularang',
  'detik-detik',
  'detik-detik lima kendaraan alami kecelakaan beruntun di cipularang'],
 [' maling di rumah mewah jakut terekam cctv bingung cari jalan kabur',
  'kocak !',
  'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur'],
 ['hampir seluas tantangan sintang siapkan generasi emas',
  'jabar ,',
  'hampir seluas jabar , ini tantangan sintang siapkan generasi emas'],
 [' diduga baku tembak di sleman , polisi',
  'viral video ini kata',
  'viral video diduga baku tembak di sleman , ini kata polisi'],
 ['waspada ada penipua

In [33]:
pc = pd.DataFrame(pairs_click)
pc

,0,1,2
0,driver di bekasi antar pesanan makanan pakai ...,viral ! ojol,viral ! driver ojol di bekasi antar pesanan ma...
1,"ada motor nyangkut bambu di sleman ,",di atas kok bisa ?,"ada motor nyangkut di atas bambu di sleman , k..."
2,pesan gamblang poyuono menolak revisi uu kpk,,pesan gamblang poyuono menolak revisi uu kpk
3,lima kendaraan alami kecelakaan beruntun di c...,detik-detik,detik-detik lima kendaraan alami kecelakaan be...
4,maling di rumah mewah jakut terekam cctv bing...,kocak !,kocak ! maling di rumah mewah jakut terekam cc...
...,...,...,...
6285,ungkap perasaan karena gelandangan bakal dike...,tamara bleszynski miris dipuji,tamara bleszynski ungkap perasaan miris karena...
6286,istana bantah jokowi tak peka pada korban kabu...,' bareng,istana bantah jokowi tak peka pada korban kabu...
6287,"rilis special ' jelang , konsep dan koreograf...",twice teaser mv ' feel comeback,twice rilis teaser mv ' feel special ' jelang ...
6288,ada niat tambah momongan tanpa ikut program ha...,,ada niat tambah momongan tanpa ikut program ha...


In [34]:
pnc = pd.DataFrame(pairs_nonclick)
pnc

,0,1,2
0,"masuk radar pilwalkot medan , menantu jokowi b...",,"masuk radar pilwalkot medan , menantu jokowi b..."
1,malaysia sudutkan ri : isu kabut asap hingga i...,,malaysia sudutkan ri : isu kabut asap hingga i...
2,"kemensos salurkan rp 7 , 3 m bagi korban kerus...",,"kemensos salurkan rp 7 , 3 m bagi korban kerus..."
3,mpr : amandemen uud 1945 tak akan melebar ke m...,,mpr : amandemen uud 1945 tak akan melebar ke m...
4,"peringati tahun baru islam , banyuwangi kembal...",,"peringati tahun baru islam , banyuwangi kembal..."
...,...,...,...
5292,"wamena papua kembali membara , kantor bupati d...",,"wamena papua kembali membara , kantor bupati d..."
5293,bmkg angkat bicara soal langit merah jambi,,bmkg angkat bicara soal langit merah jambi
5294,operasional bandara wamena dihentikan akibat r...,,operasional bandara wamena dihentikan akibat r...
5295,"asap karhutla riau mulai merambah ke nias , bm...",,"asap karhutla riau mulai merambah ke nias , bm..."


In [35]:
# for pair in pairs:
#     input_lang.addSentence(pair[0])
#     output_lang.addSentence(pair[1])
# for pair in pairs:
#     input_lang.addSentence(pair[1])
#     output_lang.addSentence(pair[0])

print("Counted words:")
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)

Counted words:
input 13924
output 13896


# Dataset:  Train, Validation, Test Split [100% Done]

In [36]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [70]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(pairs, test_size=0.3, random_state=42)
# train_data, test_data = train_test_split(pairs, test_size=0.3, random_state=42)
train_data, test_data  = train_test_split(pairs, test_size=0.2, random_state=42)
train_data, validation_data = train_test_split(train_data, test_size=0.18, random_state=42)

In [71]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(test_data)}")
print(f"Number of validation examples: {len(validation_data)}")

Number of training examples: 9840
Number of testing examples: 3000
Number of validation examples: 2160


In [72]:
print(f"{train_data[0]}")
print(test_data[0])
print(validation_data[0])

['kesaksian petugas bpdb yang bertemu beruang buas saat jalankan tugas padamkan kebakaran hutan', 'kesaksian petugas bpdb yang bertemu beruang buas saat jalankan tugas padamkan kebakaran hutan', 1]
['bekasi gandeng 3 perusahaan olah sampah jadi listrik', 'bekasi gandeng 3 perusahaan olah sampah jadi listrik', 0]
['menjajal lenovo v330-14arr , laptop 14 inci dengan amd ryzen 3', 'menjajal lenovo v330-14arr , laptop 14 inci dengan amd ryzen 3', 1]



# STAGE 3: NN Model : Encoder, Decoder, Seq2Seq [100% Done]

In [40]:
class Maxout(nn.Module):
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
        m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
        return m

In [41]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, word_vec_size, hidden_size, weight):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.embedding = nn.Embedding(input_size, word_vec_size)
        self.embedding = nn.Embedding.from_pretrained(weight)

        self.gru = nn.GRU(word_vec_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [42]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, word_vec_size, output_size, weight):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.embedding = nn.Embedding(output_size, word_vec_size)
        self.embedding = nn.Embedding.from_pretrained(weight)

        self.gru = nn.GRU(word_vec_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.maxout = Maxout(1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = self.maxout(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# STAGE 5: Improvement : Add fast text model

In [43]:
# !pip install --upgrade gensim

In [50]:
from gensim.models.fasttext import load_facebook_model

ft = load_facebook_model('fastext_model_tweet.bin')

In [51]:
# fast_text_path = "2022_thesis_styletransfer/cp_fastext_model_tweet.bin"
# ft = load_facebook_model(fast_text_path)


# fast_text_path = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/cc.id.300.bin"
# ft = load_facebook_model(fast_text_path)

# fast_text_path_mega = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/fasttext.4B.id.300.epoch5.uncased.bin"
# ft = load_facebook_model(fast_text_path_mega)

In [52]:
ft.vector_size

100

In [53]:
embedding_matrix_ft_input = np.random.random((input_lang.n_words , ft.vector_size))
embedding_matrix_ft_output = np.random.random((output_lang.n_words, ft.vector_size))

embedding_matrix_ft_input_test = np.random.random((input_lang.n_words , ft.vector_size))

# torch.zeros(max_length, encoder_a.hidden_size, device=device)

In [54]:
print(input_lang.n_words)
print(ft.vector_size)

17838
100


In [55]:
for i, word in input_lang.index2word.items():
    try:
        embedding_matrix_ft_input[i] = ft.wv[word]
    except:
        pass

for i, word in output_lang.index2word.items():
    try:
        embedding_matrix_ft_output[i] = ft.wv[word]
    except:
        pass

In [56]:
embedding_matrix_ft_input = torch.from_numpy(embedding_matrix_ft_input)
embedding_matrix_ft_output = torch.from_numpy(embedding_matrix_ft_output)

In [57]:
embedding_matrix_ft_output = embedding_matrix_ft_output.float()
embedding_matrix_ft_input = embedding_matrix_ft_input.float()

print(embedding_matrix_ft_output)

tensor([[-7.4259e-02,  1.6886e-01,  1.8216e-01,  ...,  9.6468e-02,
          2.2153e-01, -5.3748e-02],
        [-3.0592e-03, -1.5431e-03, -2.3437e-04,  ...,  1.4830e-03,
          2.1542e-03, -1.5045e-03],
        [ 9.2200e-02,  3.7329e-01, -5.2906e-01,  ..., -2.1978e-01,
         -3.4488e-01, -5.9725e-01],
        ...,
        [-1.6759e-01,  1.6140e-01, -2.9487e-01,  ...,  2.0729e-01,
          1.1030e-01, -8.4823e-02],
        [-3.5527e-01,  1.1764e-01, -1.5893e-01,  ..., -3.5791e-01,
         -8.7237e-02,  3.3565e-01],
        [-3.8605e-01,  2.8141e-01, -4.0936e-01,  ...,  2.4223e-01,
          3.4425e-01,  1.4227e-01]])


In [58]:
embedding_matrix_ft_output.size()

torch.Size([18268, 100])

# STAGE 3: Word to Index Conversion

In [52]:
#Preparing Training Data
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    # print("Indexes", indexes )
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_c_tensor = tensorFromSentence(input_lang, pair[0])
    input_a_tensor = tensorFromSentence(input_lang, pair[1])
    target_tensor = tensorFromSentence(output_lang, pair[2])
    # print(input_c_tensor)
    # print(input_a_tensor)
    return (input_c_tensor, input_a_tensor, target_tensor)

In [53]:
training_pairs = [tensorsFromPair(random.choice(pairs_click)) for i in range(1)]
training_pairs

[(tensor([[  614],
          [ 7197],
          [ 1613],
          [   34],
          [  408],
          [    6],
          [ 1289],
          [ 2056],
          [   17],
          [  249],
          [10825],
          [    1]]),
  tensor([[13],
          [ 1]]),
  tensor([[  613],
          [ 7195],
          [ 1607],
          [   33],
          [  407],
          [    6],
          [ 1282],
          [ 2050],
          [   16],
          [  248],
          [10826],
          [    1]]))]

In [54]:
#Training
MAX_LENGTH = 50

teacher_forcing_ratio = 0.5


def train(input_c_tensor, input_a_tensor, target_tensor, encoder_c, encoder_a, decoder, encoder_c_optimizer, encoder_a_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_c_hidden = encoder_c.initHidden()
    encoder_a_hidden = encoder_a.initHidden()

    # print("encoder_a_hidden", encoder_a_hidden)

    encoder_a_optimizer.zero_grad()
    encoder_c_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_c_length = input_c_tensor.size(0)
    input_a_length = input_a_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
    encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

    loss = 0

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(
            input_c_tensor[ei], encoder_c_hidden)
        encoder_c_outputs[ei] = encoder_c_output[0, 0]

    for ei in range(input_a_length):
        encoder_a_output, encoder_a_hidden = encoder_a(
            input_a_tensor[ei], encoder_a_hidden)
        encoder_a_outputs[ei] = encoder_a_output[0, 0]


    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_c_optimizer.step()
    encoder_a_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

# STAGE 3: NN Model : Training Architecture

In [55]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [56]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [73]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(pairs, test_size=0.1, random_state=42)
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(test_data)}")
print(f"{train_data[0]}")
print(test_data[0])

Number of training examples: 13500
Number of testing examples: 1500
['satu calon kades kalap dan histeris usai dibatalkan mendadak', 'satu calon kades kalap dan histeris usai dibatalkan mendadak', 0]
['bekasi gandeng 3 perusahaan olah sampah jadi listrik', 'bekasi gandeng 3 perusahaan olah sampah jadi listrik', 0]


In [57]:
def trainIters(
        encoder_a,
        encoder_c,
        decoder,
        n_iters,

        criterion,
        encoder_c_optimizer,
        encoder_a_optimizer,
        decoder_optimizer,

        print_every=1000,
        plot_every=100,
        learning_rate=0.01):

    encoder_a.train()
    encoder_c.train()
    decoder.train()

    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # encoder_c_optimizer = optim.Adadelta(encoder_c.parameters(), lr=learning_rate)
    # encoder_a_optimizer = optim.Adadelta(encoder_a.parameters(), lr=learning_rate)
    # decoder_optimizer = optim.Adadelta(decoder.parameters(), lr=learning_rate)
    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    # training_pairs = [tensorsFromPair(random.choice(pairs_click)) for i in range(n_iters)]
    training_pairs = [tensorsFromPair(train_data[i])for i in range(n_iters)]
    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    # criterion = nn.NLLLoss()

    ouput_loss_total = 0

    for iter in range(1, n_iters + 1):

        training_pair = training_pairs[iter - 1]
        input_a_tensor = training_pair[0]
        input_c_tensor = training_pair[1]
        target_tensor = training_pair[2]

        loss = train(input_c_tensor, input_a_tensor, target_tensor,
                     encoder_c, encoder_a, decoder,
                     encoder_c_optimizer, encoder_c_optimizer, decoder_optimizer,
                     criterion)

        print_loss_total += loss
        plot_loss_total += loss
        ouput_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
    #
    #     if iter % plot_every == 0:
    #         plot_loss_avg = plot_loss_total / plot_every
    #         plot_losses.append(plot_loss_avg)
    #         plot_loss_total = 0
    #
    # showPlot(plot_losses)

    return ouput_loss_total/n_iters

In [58]:
def evaluating(
        encoder_a,
        encoder_c,
        decoder,

        criterion,
        n_iters,
        plot_every : int =10,
        max_length=MAX_LENGTH
):

    encoder_a.eval()
    encoder_c.eval()
    decoder.eval()

    plot_losses = []
    plot_loss_total = 0
    loss_total_output = 0

    # training_pairs = [tensorsFromPair(random.choice(pairs_click)) for i in range(n_iters)]
    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    training_pairs = [tensorsFromPair(test_data[i])for i in range(n_iters)]

    start: float = time.time()

    with torch.no_grad():
        for iter  in  range(1, n_iters + 1):
            # taking the pair and extract the c, a, target
            training_pair = training_pairs[iter - 1]
            input_c_tensor = training_pair[0]
            input_a_tensor = training_pair[1]
            target_tensor = training_pair[2]

            # START: eval func
            # init hidden
            encoder_c_hidden = encoder_c.initHidden()
            encoder_a_hidden = encoder_a.initHidden()

            # len of sentence of c, a, t, that already; e.g input_c_length= len(input_c_tensor)
            input_c_length = input_c_tensor.size(0)
            input_a_length = input_a_tensor.size(0)
            target_length = target_tensor.size(0)

            encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
            encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

            loss = 0

            for ei in range(input_c_length):
                encoder_c_output, encoder_c_hidden = encoder_c(input_c_tensor[ei], encoder_c_hidden)
                encoder_c_outputs[ei] = encoder_c_output[0, 0]

            for ei in range(input_a_length):
                encoder_a_output, encoder_a_hidden = encoder_a(input_a_tensor[ei], encoder_a_hidden)
                encoder_a_outputs[ei] = encoder_a_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

            if use_teacher_forcing:
                # Teacher forcing: Feed the target as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    loss += criterion(decoder_output, target_tensor[di])
                    decoder_input = target_tensor[di]  # Teacher forcing

            else:
            # Without teacher forcing: use its own predictions as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    decoder_input = topi.squeeze().detach()  # detach from history as input

                    loss += criterion(decoder_output, target_tensor[di])
                    if decoder_input.item() == EOS_token:
                        break

            l = loss.item() / target_length
            # END: Eval Func

            loss_total_output += l
            plot_loss_total += l

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                # print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                #                              iter, iter / n_iters * 100, plot_loss_avg))
                plot_loss_total = 0

        # showPlot(plot_losses)

    return loss_total_output/ n_iters

In [59]:
# word_vec_size = 128
word_vec_size = ft.vector_size
hidden_size = 512

# encoder_c = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
# encoder_a = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
# decoder = DecoderRNN(hidden_size + hidden_size, word_vec_size, output_lang.n_words).to(device)

encoder_c = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)
encoder_a = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)

decoder = DecoderRNN(hidden_size + hidden_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output).to(device)

In [60]:
# trainIters(encoder_c, encoder_a, decoder, 200, print_every=100)
# len(pairs_click)
print(len(pairs) - len(pairs_click))
print(len(pairs_click))
print(len(pairs))

5297
3316
8613


In [61]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [62]:
n_t = len(train_data)
n_e = len(test_data)
N_EPOCHS = 20
best_valid_loss = float('inf')

learning_rate=0.1
# learning_rate = 0.001

encoder_c_optimizer = optim.Adadelta(encoder_c.parameters(), lr=learning_rate)
encoder_a_optimizer = optim.Adadelta(encoder_a.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adadelta(decoder.parameters(), lr=learning_rate)

# encoder_optimizer = optim.Adam(encoder1.parameters(),lr=learning_rate)
# decoder_optimizer = optim.Adam(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.Adam(style_embedder1.parameters(), lr=learning_rate)

# PATH = "delete_only_model.pt"
PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_retrieve/delete_retrive_model.pt"
MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_retrieve/"

TRAINING_DATASET_SIZE = n_t
EVALUATION_DATASET_SIZE = n_e

# criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()

plot_train_loss = []
plot_eval_loss = []

perplexity_train = []
perplexity_eval = []


for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = trainIters(
        encoder_a,
        encoder_c,
        decoder,
        TRAINING_DATASET_SIZE,

        criterion,
        encoder_c_optimizer,
        encoder_a_optimizer,
        decoder_optimizer,
        print_every=50
    )

    valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=50)

    plot_train_loss.append(train_loss)
    plot_eval_loss.append(valid_loss)


    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder_a.state_dict(), MAIN_PATH +  'encoder_a-model.pt')
        torch.save(encoder_c.state_dict(), MAIN_PATH  + 'encoder_c-model.pt')
        torch.save(decoder.state_dict(), MAIN_PATH + 'decoder-model.pt')

        torch.save({
            'epoch': epoch,
            'encoder_c_state_dict': encoder_c.state_dict(),
            'encoder_a_state_dict': encoder_a.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            'encoder_c_optimizer_state_dict': encoder_c_optimizer.state_dict(),
            'encoder_a_optimizer_state_dict': encoder_a_optimizer.state_dict(),
            'decoder_optimizer_state_dict': decoder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")

showPlot(plot_train_loss)
showPlot(plot_eval_loss)

0m 35s (- 22m 58s) (50 2%) 8.7313
0m 57s (- 18m 12s) (100 5%) 5.5482
1m 19s (- 16m 25s) (150 7%) 5.6318
1m 46s (- 15m 54s) (200 10%) 5.9161
2m 8s (- 14m 56s) (250 12%) 4.7780
2m 30s (- 14m 14s) (300 15%) 5.3833
2m 52s (- 13m 33s) (350 17%) 5.5071
3m 16s (- 13m 4s) (400 20%) 5.9503
3m 36s (- 12m 26s) (450 22%) 4.7532
3m 59s (- 11m 57s) (500 25%) 5.2679
4m 18s (- 11m 22s) (550 27%) 4.6226
4m 39s (- 10m 53s) (600 30%) 4.7787
4m 58s (- 10m 19s) (650 32%) 4.2902
5m 17s (- 9m 50s) (700 35%) 4.2425
5m 40s (- 9m 28s) (750 37%) 4.7355
6m 0s (- 9m 0s) (800 40%) 3.5813
6m 27s (- 8m 44s) (850 42%) 5.8347
6m 49s (- 8m 20s) (900 45%) 3.9243
7m 13s (- 7m 59s) (950 47%) 4.8169
7m 37s (- 7m 37s) (1000 50%) 4.5308
8m 0s (- 7m 14s) (1050 52%) 4.6374
8m 18s (- 6m 47s) (1100 55%) 3.2666
8m 37s (- 6m 22s) (1150 57%) 3.7249
9m 2s (- 6m 1s) (1200 60%) 4.7659
9m 23s (- 5m 37s) (1250 62%) 4.4472
9m 47s (- 5m 16s) (1300 65%) 3.9644
10m 13s (- 4m 55s) (1350 67%) 4.6762
10m 37s (- 4m 33s) (1400 70%) 4.0658
10m 57s

# Stage 3: Testing the DeleteRetrieve to Generate Text

In [63]:
def encoderStep(encoder_c, encoder_a, c, a, max_length=MAX_LENGTH):
    input_c_tensor = tensorFromSentence(input_lang, c)
    input_a_tensor = tensorFromSentence(input_lang, a)

    input_c_length = input_c_tensor.size()[0]
    input_a_length = input_a_tensor.size()[0]

    encoder_c_hidden = encoder_c.initHidden()
    encoder_a_hidden = encoder_a.initHidden()

    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
    encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(input_c_tensor[ei],
                                                       encoder_c_hidden)
        encoder_c_outputs[ei] += encoder_c_output[0, 0]


    for ei in range(input_a_length):
        encoder_a_output, encoder_a_hidden = encoder_a(input_a_tensor[ei],
                                                       encoder_a_hidden)
        encoder_a_outputs[ei] += encoder_a_output[0, 0]



    return torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

In [64]:
def get_c_embedding(encoder_c, c, max_length=MAX_LENGTH):
    input_c_tensor = tensorFromSentence(input_lang, c)

    input_c_length = input_c_tensor.size()[0]

    encoder_c_hidden = encoder_c.initHidden()

    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(input_c_tensor[ei],
                                                       encoder_c_hidden)
        encoder_c_outputs[ei] += encoder_c_output[0, 0]

    return encoder_c_hidden

In [65]:
ALPHA = 0.7

class Node:
    def __init__(self, p_t, i, d_hidden, lvl, node_parent=None):
        self.p_t = p_t
        self.i = i
        self.d_hidden = d_hidden
        self.lvl = lvl
        self.node_parent = node_parent

        self.p_sentence = self.sentenceProb()

    def prepareToDecode(self):
        d_in = self.i.squeeze()#.detatch()
        return d_in, self.d_hidden

    def sentenceProb(self):
        if self.node_parent == None:
            return self.p_t
        return self.node_parent.sentenceProb() + self.p_t #addition because of log

    def normProb(self):
        #Normalised probability
        #         return 1/((self.lvl+1)**ALPHA) * self.sentenceProb()
        #perplexity
        return torch.exp(self.sentenceProb()) ** (-1/(self.lvl+1))

    def getToken(self):
        return output_lang.index2word[self.i.item()]

    def getTokens(self):
        token = [self.getToken()]

        if(self.node_parent == None):
            return token
        else:
            return self.node_parent.getTokens() + token

    def getSentence(self):
        return ' '.join(self.getTokens())

In [66]:
BEAM_WIDTH = 10

def evaluate(encoder_c, encoder_a, decoder, c, a, max_length=MAX_LENGTH):
    with torch.no_grad():
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoderStep(encoder_c, encoder_a, c, a)
        decoded_words = []

        #BEAM SEARCH

        nodes = []
        finished_nodes = []
        for di in range(max_length):
            if di == 0:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                top_ps, top_is = decoder_output.data.topk(BEAM_WIDTH)
                top_ps = top_ps.view(-1)
                top_is = top_is.view(-1)
                for index in range(len(top_is)):
                    p_t = top_ps[index]
                    i = top_is[index]
                    nodes.append(Node(p_t, i, decoder_hidden, di))
            else:
                prev_nodes = [x for x in nodes if x.lvl == di-1]
                prev_nodes = sorted(prev_nodes, key=lambda x: x.p_sentence.item(), reverse=True)
                prev_nodes = prev_nodes[:BEAM_WIDTH]
                nodes = []
                for node in prev_nodes:
                    decoder_output, decoder_hidden = decoder(*node.prepareToDecode())
                    top_ps, top_is = decoder_output.data.topk(BEAM_WIDTH)
                    top_ps = top_ps.view(-1)
                    top_is = top_is.view(-1)

                    for index in range(len(top_is)):
                        p_t = top_ps[index]
                        i = top_is[index]
                        child_node = Node(p_t, i, decoder_hidden, di, node)

                        if i.item() == EOS_token:
                            finished_nodes.append(child_node)
                        else:
                            nodes.append(child_node)

        final_node = sorted(finished_nodes, key=lambda x: x.p_sentence.item(), reverse=True)[0]
        return final_node.getTokens()

# evaluate(encoder_c, encoder_a, decoder, "the food", "great")

In [67]:
def evaluateWithoutBeamSearch(encoder_c, encoder_a, decoder, c, a, max_length=MAX_LENGTH):
    with torch.no_grad():
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoderStep(encoder_c, encoder_a, c, a)
        decoded_words = []

        #BEAM SEARCH
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            topv, topi = F.softmax(decoder_output).data.topk(1)

            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words
# ' '.join(evaluateWithoutBeamSearch(encoder_c, encoder_a, decoder, "the food is", "great"))

In [68]:
def evaluateRandomly(encoder_c, encoder_a, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder_c, encoder_a, decoder, pair[0], pair[1])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [69]:
evaluateRandomly(encoder_c, encoder_a, decoder)

> agus rahardjo pastikan aktivitas kpk berjalan seperti biasa
= 
< , EOS

> haris azhar kritik menkumham soal gel ngan didenda rp 1 juta : gembel enggak nonton pidato 
= anda
< , EOS

> ishlah , kunci pengurusan anak yatim menurut alquran
= apa kata
< , EOS

>  mirip , polisi tilang ayah kandungnya
= ini ternyata
< , EOS

> anies cabut kasasi perkara lahan sodetan ciliwung era ahok di ma
= 
< , EOS

> tinggal di lingkungan sineas , tetangga joko anwar ada tara basro sampai dimas anggara
= 
< , EOS

> 850 personel polisi siap amankan itdbi 2019
= 
< , EOS

> menu sehat dari sdn sebabkan balita meninggal di koja 
= ?
< , EOS

>  wartawan rekam tindakan represif aparat polisi jadi 
= viral , sikap ini sorotan
< , EOS

> tiga bulan , 400 tangki air dikirim ke warga lamongan atasi kekeringan
= 
< , EOS



In [70]:
def DeleteAndRetrieve(sentence, style):
    c_src = separate(sentence, 0)['c']
    a_tgt = ' '.join(retrieve(sentence, not style)['a'])
    return ' '.join(evaluate(encoder_c, encoder_a, decoder, c_src, a_tgt))

# STAGE 4: ADDITIONAL: FULL COPY ONLY ONE EPOCH

In [71]:
# class Maxout(nn.Module):
#     def __init__(self, pool_size):
#         super().__init__()
#         self._pool_size = pool_size
#
#     def forward(self, x):
#         assert x.shape[1] % self._pool_size == 0, \
#             'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
#         m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
#         return m

In [72]:
# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, word_vec_size, hidden_size):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#
#         self.embedding = nn.Embedding(input_size, word_vec_size)
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#
#     def forward(self, input, hidden):
#         embedded = self.embedding(input).view(1, 1, -1)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

In [73]:
# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding(output_size, word_vec_size)
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)
#
#     def forward(self, input, hidden):
#         output = self.embedding(input).view(1, 1, -1)
#         output = self.maxout(output)
#         output, hidden = self.gru(output, hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

In [74]:
# #Preparing Training Data
# def indexesFromSentence(lang, sentence):
#     return [lang.word2index[word] for word in sentence.split(' ')]
#
#
# def tensorFromSentence(lang, sentence):
#     indexes = indexesFromSentence(lang, sentence)
#     indexes.append(EOS_token)
#     return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)
#
#
# def tensorsFromPair(pair):
#     input_c_tensor = tensorFromSentence(input_lang, pair[0])
#     input_a_tensor = tensorFromSentence(input_lang, pair[1])
#     target_tensor = tensorFromSentence(output_lang, pair[2])
#     return (input_c_tensor, input_a_tensor, target_tensor)

In [75]:
# #Training
# MAX_LENGTH = 50
#
# teacher_forcing_ratio = 0.5
#
#
# def train(input_c_tensor, input_a_tensor, target_tensor, encoder_c, encoder_a, decoder, encoder_c_optimizer, encoder_a_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
#     encoder_c_hidden = encoder_c.initHidden()
#     encoder_a_hidden = encoder_a.initHidden()
#
#     encoder_a_optimizer.zero_grad()
#     encoder_c_optimizer.zero_grad()
#
#     decoder_optimizer.zero_grad()
#
#     input_c_length = input_c_tensor.size(0)
#     input_a_length = input_a_tensor.size(0)
#
#     target_length = target_tensor.size(0)
#
#     encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
#     encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)
#
#     loss = 0
#
#     for ei in range(input_c_length):
#         encoder_c_output, encoder_c_hidden = encoder_c(
#             input_c_tensor[ei], encoder_c_hidden)
#         encoder_c_outputs[ei] = encoder_c_output[0, 0]
#
#     for ei in range(input_a_length):
#         encoder_a_output, encoder_a_hidden = encoder_a(
#             input_a_tensor[ei], encoder_a_hidden)
#         encoder_a_outputs[ei] = encoder_a_output[0, 0]
#
#
#     decoder_input = torch.tensor([[SOS_token]], device=device)
#
#     decoder_hidden = torch.cat((encoder_c_hidden, encoder_a_hidden), 2)
#
#     use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
#
#     if use_teacher_forcing:
#         # Teacher forcing: Feed the target as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(
#                 decoder_input, decoder_hidden)
#             loss += criterion(decoder_output, target_tensor[di])
#             decoder_input = target_tensor[di]  # Teacher forcing
#
#     else:
#         # Without teacher forcing: use its own predictions as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(
#                 decoder_input, decoder_hidden)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.squeeze().detach()  # detach from history as input
#
#             loss += criterion(decoder_output, target_tensor[di])
#             if decoder_input.item() == EOS_token:
#                 break
#
#     loss.backward()
#
#     encoder_c_optimizer.step()
#     encoder_a_optimizer.step()
#     decoder_optimizer.step()
#
#     return loss.item() / target_length

In [76]:
# import time
# import math
#
# def asMinutes(s):
#     m = math.floor(s / 60)
#     s -= m * 60
#     return '%dm %ds' % (m, s)
#
#
# def timeSince(since, percent):
#     now = time.time()
#     s = now - since
#     es = s / (percent)
#     rs = es - s
#     return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [77]:
# import matplotlib.pyplot as plt
# plt.switch_backend('agg')
# import matplotlib.ticker as ticker
# import numpy as np
#
#
# def showPlot(points):
#     plt.figure()
#     fig, ax = plt.subplots()
#     # this locator puts ticks at regular intervals
#     loc = ticker.MultipleLocator(base=0.2)
#     ax.yaxis.set_major_locator(loc)
#     plt.plot(points)

In [78]:
# def trainIters(encoder_a, encoder_c, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
#     start = time.time()
#     plot_losses = []
#     print_loss_total = 0  # Reset every print_every
#     plot_loss_total = 0  # Reset every plot_every
#
#     encoder_c_optimizer = optim.Adadelta(encoder_c.parameters(), lr=learning_rate)
#     encoder_a_optimizer = optim.Adadelta(encoder_a.parameters(), lr=learning_rate)
#     decoder_optimizer = optim.Adadelta(decoder.parameters(), lr=learning_rate)
#     training_pairs = [tensorsFromPair(random.choice(pairs))
#                       for i in range(n_iters)]
#     criterion = nn.NLLLoss()
#
#     for iter in range(1, n_iters + 1):
#         training_pair = training_pairs[iter - 1]
#         input_a_tensor = training_pair[0]
#         input_c_tensor = training_pair[1]
#
#         target_tensor = training_pair[2]
#         loss = train(input_c_tensor, input_a_tensor, target_tensor, encoder_c, encoder_a,
#                      decoder, encoder_c_optimizer, encoder_c_optimizer, decoder_optimizer, criterion)
#         print_loss_total += loss
#         plot_loss_total += loss
#
#         if iter % print_every == 0:
#             print_loss_avg = print_loss_total / print_every
#             print_loss_total = 0
#             print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
#                                          iter, iter / n_iters * 100, print_loss_avg))
#
#         if iter % plot_every == 0:
#             plot_loss_avg = plot_loss_total / plot_every
#             plot_losses.append(plot_loss_avg)
#             plot_loss_total = 0
#
#     showPlot(plot_losses)

In [79]:
# word_vec_size = 128
# hidden_size = 512
# encoder_c = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
# encoder_a = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
#
# decoder = DecoderRNN(hidden_size + hidden_size, word_vec_size, output_lang.n_words).to(device)

In [80]:
# trainIters(encoder_c, encoder_a, decoder, 8613, print_every=200)

# STAGE 5: DELETE ONLY FUNCTION

In [59]:
# Resources: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# Had to modify & adapt most of the code in the tutorial since this isn't translation & data preprocessing is different

SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [60]:
param_t = 4

def prepareData():
    input_lang = Lang("input")
    output_lang = Lang("output")

    pairs = []
    pairs_pos = []
    pairs_neg = []

    for sentence in clickbait_l_c:
        comp = [get_c(sentence, 1, param_t), sentence, 1]
        pairs_neg.append(comp)
        pairs.append(comp)

    for sentence in nonclickbait_l_c:
        comp = [get_c(sentence, 0, param_t), sentence, 0]
        pairs_pos.append(comp)
        pairs.append(comp)

    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs, pairs_pos, pairs_neg

In [32]:
input_lang, output_lang, pairs, pairs_nonclick, pairs_nonclick  = prepareData()

/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_2115/4054946038.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)


Counted words:
input 17838
output 18268


In [63]:
# for pair in pairs:
#     input_lang.addSentence(pair[0])
#     output_lang.addSentence(pair[1])
# for pair in pairs:
#     input_lang.addSentence(pair[1])
#     output_lang.addSentence(pair[0])

print("Counted words:")
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)

Counted words:
input 17838
output 18268


In [64]:
# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, word_vec_size, hidden_size, weight, dropout):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#
#         # self.embedding = nn.Embedding(input_size, word_vec_size)
#         self.embedding = nn.Embedding.from_pretrained(weight)
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.dropout = nn.Dropout(dropout)
#
#     def forward(self, input, hidden):
#         embedded = self.embedding(input).view(1, 1, -1)
#         embedded = self.dropout(embedded)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

# encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input, ENC_DROPOUT).to(device)


class EncoderRNN(nn.Module):
    def __init__(self, input_size, word_vec_size, hidden_size, weight):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding.from_pretrained(weight)
        self.gru = nn.GRU(word_vec_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, word_vec_size, hidden_size, weight):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.layer = 8
#
#         # self.embedding = nn.Embedding(input_size, word_vec_size)
#         self.embedding = nn.Embedding.from_pretrained(weight)
#         self.gru = nn.GRU(word_vec_size, hidden_size, self.layer)
#
#     def forward(self, input, hidden):
#         embedded = self.embedding(input).view(1, 1, -1)
#         output = embedded
#         output, hidden = self.gru(output, hidden)
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

In [65]:
class StyleEmbedder(nn.Module):
    def __init__(self, num_styles, dimensions):
        super(StyleEmbedder, self).__init__()
        self.dimensions = dimensions
        self.embedding = nn.Embedding(num_styles, dimensions)

    def forward(self, input):
        embedded = self.embedding(input).view(1, 1, -1)
        return embedded

In [66]:
class Maxout(nn.Module):
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
        m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
        return m

In [67]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, word_vec_size, output_size, weight):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding.from_pretrained(weight)

        self.gru = nn.GRU(word_vec_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.maxout = Maxout(1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = self.maxout(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size, weight, dropout):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         # self.layer = 8;
#
#         # self.embedding = nn.Embedding(output_size, word_vec_size)
#         self.embedding = nn.Embedding.from_pretrained(weight)
#
#         # self.gru = nn.GRU(word_vec_size, hidden_size, self.layer)
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)
#         self.dropout = nn.Dropout(dropout)
#
#     def forward(self, input, hidden):
#         output = self.embedding(input).view(1, 1, -1)
#         output = self.maxout(output)
#         output, hidden = self.gru(output, hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)


# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size, weight):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#
#         # self.embedding = nn.Embedding(output_size, word_vec_size)
#         self.embedding = nn.Embedding.from_pretrained(weight)
#
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)
#         self.attention = nn.Linear(hidden_size + word_vec_size, hidden_size)
#
#
#     def forward(self, input, hidden):
#         # Calculate the attention weights
#
#         print("input type", type(input) )
#         print("hidden type", type(hidden) )
#
#         print("input", input.size())
#         print("hidden", hidden.size())
#
#         test2 = (input.view(1, 1, -1), hidden)
#
#         print("input_view", len(test2))
#
#         attn_weights = F.softmax(self.attention(torch.cat(test2, 1)))
#         # attn_weights = F.softmax(self.attention(torch.cat((input, hidden), 1)))
#
#         # Create a weighted sum of the input and hidden state using the attention weights as coefficients
#         context = attn_weights * input + (1 - attn_weights) * hidden
#
#         # Pass the weighted sum through the GRU layer along with the embedded input to produce the decoder's output
#         output, hidden = self.gru(torch.cat((context, self.embedding(input).view(1, 1, -1)), 2), hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)



#
# MAX_LENGTH = 20
# class AttnDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size, weight, dropout_p=0.1, max_length=MAX_LENGTH):
#         super(AttnDecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.output_size = output_size
#         self.dropout_p = dropout_p
#         self.max_length = max_length
#
#         # hidden_size = word_vec_size;
#         # self.embedding = nn.Embedding.from_pretrained(weight)
#         self.embedding = nn.Embedding(self.output_size, self.hidden_size)
#         self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
#         self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
#         self.dropout = nn.Dropout(self.dropout_p)
#         self.gru = nn.GRU(self.hidden_size, self.hidden_size)
#         self.out = nn.Linear(self.hidden_size, self.output_size)
#
#     def forward(self, input, hidden, encoder_outputs):
#         embedded = self.embedding(input).view(1, 1, -1)
#         embedded = self.dropout(embedded)
#
#         attn_weights = F.softmax(
#             self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
#         attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
#
#         output = torch.cat((embedded[0], attn_applied[0]), 1)
#         output = self.attn_combine(output).unsqueeze(0)
#
#         output = F.relu(output)
#         output, hidden = self.gru(output, hidden)
#
#         output = F.log_softmax(self.out(output[0]), dim=1)
#         return output, hidden, attn_weights
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

#
# class Attention(nn.Module):
#     def __init__(self, hidden_size):
#         super(Attention, self).__init__()
#         self.hidden_size = hidden_size
#         self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
#         self.v = nn.Parameter(torch.rand(hidden_size))
#         stdv = 1. / math.sqrt(self.v.size(0))
#         self.v.data.normal_(mean=0, std=stdv)
#
#     def forward(self, hidden, encoder_outputs):
#         timestep = encoder_outputs.size(0)
#         h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
#         encoder_outputs = encoder_outputs.transpose(0, 1)
#         attn_energies = self.score(h, encoder_outputs)
#         return F.softmax(attn_energies, dim=1).unsqueeze(1)
#
#     def score(self, hidden, encoder_outputs):
#
#         print("hidden", hidden)
#         print("encoder_outputs", encoder_outputs)
#
#         energy = F.relu(self.attn(torch.cat([hidden, encoder_outputs], 2)))
#         energy = energy.transpose(1, 2)
#         v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
#         energy = torch.bmm(v, energy)
#         return energy.squeeze(1)
#
#


# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size, weight):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.embedding = nn.Embedding.from_pretrained(weight)
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)
#         self.attn = Attention(hidden_size)
#
#     def forward(self, input, hidden, encoder_outputs):
#         output = self.embedding(input).view(1, 1, -1)
#         output = self.maxout(output)
#         output, hidden = self.gru(output, hidden)
#         attn_weights = self.attn(output, encoder_outputs)
#         context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
#         output = output.squeeze(0)
#         context = context.squeeze(1)
#         output = self.softmax(self.out(torch.cat([output, context], 1)))
#         return output, hidden, attn_weights
#
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)


# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, word_vec_size, output_size, weight):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#
#         self.embedding = nn.Embedding.from_pretrained(weight)
#
#         # Add an attention layer
#         self.attention = nn.Linear(hidden_size + word_vec_size, hidden_size)
#
#         self.gru = nn.GRU(word_vec_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)
#         self.maxout = Maxout(1)
#
#     def forward(self, input, hidden, encoder_outputs):
#         # Calculate the attention weights
#         attn_weights = F.softmax(self.attention(torch.cat((input, hidden), 1)))
#         # attn_weights = F.softmax(self.attention(torch.cat((input.unsqueeze(1), hidden), 1)))
#
#
#         # Create a weighted sum of the encoder outputs using the attention weights as coefficients
#         context = attn_weights.unsqueeze(0).bmm(encoder_outputs.unsqueeze(0))
#
#         # Pass the weighted sum through the GRU layer along with the embedded input to produce the decoder's output
#         output, hidden = self.gru(torch.cat((context, self.embedding(input).view(1, 1, -1)), 2), hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden


In [68]:
#Preparing Training Data
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorFromStyle(style):
    #     one_hot_encoded_style = []
    #     if style:
    #         one_hot_encoded_style = [1,0]
    #     else:
    #         one_hot_encoded_style = [0,1]
    return torch.tensor(style, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    style_tensor = tensorFromStyle(pair[2])
    return (input_tensor, style_tensor, target_tensor)  #add style_tensor

In [69]:
#Trainin

teacher_forcing_ratio = 0.5

MAX_LENGTH = 50


def train(
        input_tensor,
        style_tensor,
        target_tensor,

        encoder,
        style_embedder,
        decoder,

        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,

        criterion,
        max_length=MAX_LENGTH):


    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    style_embedder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    #calculate style embedding
    style_embedding = style_embedder(style_tensor)

    decoder_hidden = torch.cat((encoder_hidden, style_embedding), 2) #TODO: concatenate style embedding

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    style_embedder_optimizer.step()
    decoder_optimizer.step()


    return loss.item() / target_length

In [70]:
# # chatgpt;
# # MAX_LENGTH = 20
# MAX_LENGTH = 50
# #Trainin
# teacher_forcing_ratio = 0.5
#
# def train(
#         input_tensor,
#         style_tensor,
#         target_tensor,
#
#         encoder,
#         style_embedder,
#         decoder,
#
#         encoder_optimizer,
#         style_embedder_optimizer,
#         decoder_optimizer,
#
#         criterion,
#         max_length=MAX_LENGTH):
#
#
#     encoder_hidden = encoder.initHidden()
#
#     encoder_optimizer.zero_grad()
#     decoder_optimizer.zero_grad()
#     style_embedder_optimizer.zero_grad()
#
#     input_length = input_tensor.size(0)
#     target_length = target_tensor.size(0)
#
#     encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
#
#     loss = 0
#
#     # Encode the input sequence using the encoder
#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(
#             input_tensor[ei], encoder_hidden)
#         encoder_outputs[ei] = encoder_output[0, 0]
#
#     # Compute the embedded style representation
#     style_embedding = style_embedder(style_tensor)
#
#     # Initialize the decoder input and hidden state
#     decoder_input = torch.tensor([[SOS_token]], device=device)
#
#     # Concatenate encoder_hidden and style_embedding along the last dimension
#     decoder_hidden = torch.cat((encoder_hidden, style_embedding), 2)
#     # decoder_hidden = torch.cat((encoder_hidden[0], style_embedding), 2)
#
#
#     # Use teacher forcing if the random value is less than the teacher forcing ratio
#     use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
#
#     # Decode the input sequence using the decoder and compute the loss
#     if use_teacher_forcing:
#         # Teacher forcing: Feed the target as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#             loss += criterion(decoder_output, target_tensor[di])
#             decoder_input = target_tensor[di]  # Teacher forcing
#
#
#     else:
#         # Without teacher forcing: use its own predictions as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.squeeze().detach()  # detach from history as input
#
#             loss += criterion(decoder_output, target_tensor[di])
#             if decoder_input.item() == EOS_token:
#                 break
#
#     # Backpropagate the loss and update the model weights
#     loss.backward()
#     encoder_optimizer.step()
#     style_embedder_optimizer.step()
#     decoder_optimizer.step()
#
#     return loss.item() / target_length
#
#


In [71]:
# #Trainin
#
# teacher_forcing_ratio = 0.5
#
#
# def train(
#         input_tensor,
#         style_tensor,
#         target_tensor,
#
#         encoder,
#         style_embedder,
#         decoder,
#
#         encoder_optimizer,
#         style_embedder_optimizer,
#         decoder_optimizer,
#
#         criterion,
#         max_length=MAX_LENGTH):
#
#
#     encoder_hidden = encoder.initHidden()
#
#     encoder_optimizer.zero_grad()
#     decoder_optimizer.zero_grad()
#     style_embedder_optimizer.zero_grad()
#
#     input_length = input_tensor.size(0)
#     target_length = target_tensor.size(0)
#
#     encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
#
#     loss = 0
#
#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(
#             input_tensor[ei], encoder_hidden)
#         encoder_outputs[ei] = encoder_output[0, 0]
#
#     # for ei in range(input_length):
#     #     encoder_output, encoder_hidden = encoder( input_tensor[ei], encoder_hidden)
#     #     encoder_outputs[ei] = encoder_output[0, 0]
#
#
#     decoder_input = torch.tensor([[SOS_token]], device=device)
#
#     # decoder_hidden = encoder_hidden
#
#     #calculate style embedding
#     style_embedding = style_embedder(style_tensor)
#
#     decoder_hidden = torch.cat((encoder_hidden, style_embedding), 2) #TODO: concatenate style embedding
#
#     use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
#
#     # if use_teacher_forcing:
#     #     # Teacher forcing: Feed the target as the next input
#     #     for di in range(target_length):
#     #         decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
#     #         loss += criterion(decoder_output, target_tensor[di])
#     #         decoder_input = target_tensor[di]  # Teacher forcing
#     #
#     # else:
#     # # Without teacher forcing: use its own predictions as the next input
#     #     for di in range(target_length):
#     #         decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
#     #         topv, topi = decoder_output.topk(1)
#     #         decoder_input = topi.squeeze().detach()  # detach from history as input
#     #
#     #         loss += criterion(decoder_output, target_tensor[di])
#     #         if decoder_input.item() == EOS_token:
#     #             break
#     # #
#     if use_teacher_forcing:
#         # Teacher forcing: Feed the target as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#             loss += criterion(decoder_output, target_tensor[di])
#             decoder_input = target_tensor[di]  # Teacher forcing
#
#     else:
#         # Without teacher forcing: use its own predictions as the next input
#         for di in range(target_length):
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.squeeze().detach()  # detach from history as input
#
#             loss += criterion(decoder_output, target_tensor[di])
#             if decoder_input.item() == EOS_token:
#                 break
#
#     loss.backward()
#     encoder_optimizer.step()
#     style_embedder_optimizer.step()
#     decoder_optimizer.step()
#
#
#     return loss.item() / target_length

In [72]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [73]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(pairs, test_size=0.05, random_state=42)
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(test_data)}")
print(f"{train_data[0]}")
print(test_data[0])

Number of training examples: 14250
Number of testing examples: 750
[' dan jessi terancam kena pasal seksual celana dalam ', 'hyuna dan jessi terancam kena pasal seksual gara-gara umbar celana dalam ?', 1]
['bekasi gandeng 3 perusahaan olah sampah jadi listrik', 'bekasi gandeng 3 perusahaan olah sampah jadi listrik', 0]


In [74]:
def trainIters(
        encoder,
        style_embedder,
        decoder,

        criterion,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,

        n_iters,
        print_every=1000,
        plot_every=100):

    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder.train()
    style_embedder.train()
    decoder.train()

    # training_pairs = [tensorsFromPair(random.choice(pairs))for i in range(n_iters)]
    training_pairs = [tensorsFromPair(train_data[i])for i in range(n_iters)]
    # criterion = nn.NLLLoss()

    ouput_loss_total = 0

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        style_tensor = training_pair[1]
        target_tensor = training_pair[2]

        loss = train(input_tensor, style_tensor, target_tensor, encoder, style_embedder,
                     decoder, encoder_optimizer, style_embedder_optimizer, decoder_optimizer, criterion)

        print_loss_total += loss
        plot_loss_total += loss
        ouput_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    return ouput_loss_total/n_iters


In [75]:
def evaluating(
        encoder,
        style_embedder,
        decoder,

        criterion,
        n_iters,
        plot_every : int =10,
        max_length=MAX_LENGTH
):

    encoder.eval()
    style_embedder.eval()
    decoder.eval()

    plot_losses = []
    plot_loss_total = 0
    loss_total_output = 0

    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    training_pairs = [tensorsFromPair(train_data[i])for i in range(n_iters)]

    start: float = time.time()

    with torch.no_grad():
        for iter  in  range(1, n_iters + 1):
            # taking the pair and extract the c, a, target
            training_pair = training_pairs[iter - 1]
            input_tensor = training_pair[0]
            style_tensor = training_pair[1]
            target_tensor = training_pair[2]

            encoder_hidden = encoder.initHidden()

            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            style_embedder_optimizer.zero_grad()


            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            # for ei in range(input_length):
            #     encoder_output, encoder_hidden = encoder(
            #         input_tensor[ei], encoder_hidden)
            #     encoder_outputs[ei] = encoder_output[0, 0]

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(
                    input_tensor[ei], encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]


            decoder_input = torch.tensor([[SOS_token]], device=device)

            #calculate style embedding
            style_embedding = style_embedder(style_tensor)

            decoder_hidden = torch.cat((encoder_hidden, style_embedding), 2) #TODO: concatenate style embedding

            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

            if use_teacher_forcing:
                # Teacher forcing: Feed the target as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    loss += criterion(decoder_output, target_tensor[di])
                    decoder_input = target_tensor[di]  # Teacher forcing

            else:
                # Without teacher forcing: use its own predictions as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    decoder_input = topi.squeeze().detach()  # detach from history as input

                    loss += criterion(decoder_output, target_tensor[di])
                    if decoder_input.item() == EOS_token:
                        break

            # if use_teacher_forcing:
            #     # Teacher forcing: Feed the target as the next input
            #     for di in range(target_length):
            #         decoder_output, decoder_hidden, decoder_attention = decoder(
            #             decoder_input, decoder_hidden, encoder_outputs)
            #         loss += criterion(decoder_output, target_tensor[di])
            #         decoder_input = target_tensor[di]  # Teacher forcing
            #
            # else:
            #     # Without teacher forcing: use its own predictions as the next input
            #     for di in range(target_length):
            #         decoder_output, decoder_hidden, decoder_attention = decoder(
            #             decoder_input, decoder_hidden, encoder_outputs)
            #         topv, topi = decoder_output.topk(1)
            #         decoder_input = topi.squeeze().detach()  # detach from history as input
            #
            #         loss += criterion(decoder_output, target_tensor[di])
            #         if decoder_input.item() == EOS_token:
            #             break


            l = loss.item() / target_length

            loss_total_output += l
            plot_loss_total += l

    return loss_total_output/ n_iters

In [76]:
# word_vec_size = 128
word_vec_size = ft.vector_size

hidden_size = 512
style_vec_size = 128

# hidden_size = ft.vector_size

# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

# encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input ).to(device)
# decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words).to(device)

# encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input, ENC_DROPOUT).to(device)
# decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output, DEC_DROPOUT).to(device)

# decoder1 = AttnDecoderRNN(hidden_size + style_vec_size, output_lang.n_words, embedding_matrix_ft_output,  dropout_p=0.1).to(device)


# encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)
# decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output).to(device)
#
# style_embedder1 = StyleEmbedder(2, style_vec_size).to(device)

In [77]:
# word_vec_size = 128
word_vec_size = ft.vector_size

hidden_size = 512
style_vec_size = 128

encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)
decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output).to(device)

style_embedder1 = StyleEmbedder(2, style_vec_size).to(device)

In [78]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [79]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [80]:
n_t = len(train_data)
n_e = len(test_data)
N_EPOCHS = 1
best_valid_loss = float('inf')

# learning_rate=0.1
# learning_rate = 0.001

# encoder_optimizer = optim.Adadelta(encoder1.parameters(), lr=learning_rate)
# decoder_optimizer = optim.Adadelta(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.Adadelta(style_embedder1.parameters(), lr=learning_rate)

# encoder_optimizer = optim.Adam(encoder1.parameters(),lr=learning_rate)
# decoder_optimizer = optim.Adam(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.Adam(style_embedder1.parameters(), lr=learning_rate)

# encoder_optimizer = optim.SGD(encoder1.parameters(), lr=learning_rate)
# decoder_optimizer = optim.SGD(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.SGD(style_embedder1.parameters(), lr=learning_rate)
#
# encoder_optimizer = optim.RMSprop(encoder1.parameters(), lr=learning_rate)
# decoder_optimizer = optim.RMSprop(decoder1.parameters(), lr=learning_rate)
# style_embedder_optimizer = optim.RMSprop(style_embedder1.parameters(), lr=learning_rate)


encoder_learning_rate = 0.001
decoder_learning_rate = 0.002
style_embedder_learning_rate = 0.003

# Use different optimization algorithms for each optimizer
encoder_optimizer = optim.Adam(encoder1.parameters(), lr=encoder_learning_rate)
decoder_optimizer = optim.SGD(decoder1.parameters(), lr=decoder_learning_rate)
style_embedder_optimizer = optim.Adam(style_embedder1.parameters(), lr=style_embedder_learning_rate)

# Use a learning rate schedule to adjust the learning rates during training
encoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(encoder_optimizer, 'min')
decoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(decoder_optimizer, 'min')
style_embedder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(style_embedder_optimizer, 'min')

criterion1 = nn.CrossEntropyLoss()

TRAINING_DATASET_SIZE = n_t
EVALUATION_DATASET_SIZE = n_e

# PATH = "delete_only_model.pt"
PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/delete_only_model.pt"
MAIN_PATH = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/checkpoint/"

# criterion1 = nn.NLLLoss()


plot_train_losses = []
plot_eval_losses = []
perplexity_train = []
perplexity_eval = []

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = trainIters(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,
        TRAINING_DATASET_SIZE,
        print_every=200
    )

    # valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=20)

    valid_loss = evaluating(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        EVALUATION_DATASET_SIZE,
        plot_every=100)

    encoder_scheduler.step(valid_loss)
    decoder_scheduler.step(valid_loss)
    style_embedder_scheduler.step(valid_loss)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder1.state_dict(), MAIN_PATH + 'encoder1-model.pt')
        torch.save(style_embedder1.state_dict(),MAIN_PATH + 'style_embedder1-model.pt')
        torch.save(decoder1.state_dict(),MAIN_PATH +  'decoder1-model.pt')

        torch.save({
            'epoch': epoch,
            'encoder1_state_dict': encoder1.state_dict(),
            'style_embedder1-model': style_embedder1.state_dict(),
            'decoder1_state_dict': decoder1.state_dict(),
            'encoder1_optimizer_state_dict': encoder_optimizer.state_dict(),
            'decoder1_optimizer_state_dict': decoder_optimizer.state_dict(),
            'style_embedder1_optimizer_state_dict': style_embedder_optimizer.state_dict(),
            'loss': train_loss,
        }, PATH)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(encoder1.state_dict(), 'encoder1-model.pt')
    #     torch.save(style_embedder1.state_dict(), 'style_embedder1-model.pt')
    #     torch.save(decoder1.state_dict(), 'decoder1-model.pt')

    plot_train_losses.append(train_loss)
    plot_eval_losses.append(valid_loss)


    perplexity_train.append(math.exp(train_loss))
    perplexity_eval.append(math.exp(valid_loss))


    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")

showPlot(plot_train_losses)
showPlot(plot_eval_losses)

KeyboardInterrupt: 

# Create Clickbait title for Nonclickbait dataset

In [ ]:
pairs

# BLEU Evaluation Metrics